In [1]:
import warnings
from hvac import Quantity
from hvac.fluids import (
    HumidAir, Fluid, CoolPropWarning,
    STANDARD_TEMPERATURE, STANDARD_PRESSURE
)
from hvac.heat_exchanger.recuperator.fintube.continuous_fin import (
    PlainFinTubeAirToWaterCounterFlowHeatExchanger as AirCoil
)

from hvac.air_conditioning import AirConditioningProcess

warnings.filterwarnings('ignore', category=CoolPropWarning)

Q_ = Quantity

Water = Fluid('Water')

Air = Fluid('Air')
standard_air = Air(T=STANDARD_TEMPERATURE, P=STANDARD_PRESSURE)

# Air-to-Water Cooling Coil (Part 2)

Consider the same air-cooling coil from the previous notebook (part 1) installed in a space under the assumption of (1) a fixed, constant space load and (2) without thermostat control. The mass flow rate of air and water through the air-cooling coil, and the entering water temperature are known and assumed constant.

**Problem:**<br>
We want to determine the steady-state operating conditions of the air-cooling coil and the state of the space air when thermal equilibrium has been reached, i.e., when the cooling capacity of the air-cooling coil balances the fixed space load.

First, we define the air-cooling coil again:

In [2]:
air_coil = AirCoil(
    width=Q_(900, 'mm'),
    height=Q_(180, 'mm'),
    num_rows=3,
    pitch_trv=Q_(25.4, 'mm'),
    pitch_lon=Q_(22.0, 'mm'),
    d_i=Q_(8.422, 'mm'),
    d_o=Q_(10.2, 'mm'),
    t_fin=Q_(0.3302, 'mm'),
    fin_density=1 / Q_(3.175, 'mm'),
    num_circuits=2
)

The fixed operating conditions of the air-cooling coil are set:
- the volume and mass flow rate of air through the air-cooling coil
- the entering water temperature
- the volume and mass flow rate of water through the air-cooling coil

In [3]:
air_V_dot = Q_(850.0, 'm ** 3 / hr')
air_m_dot = standard_air.rho * air_V_dot

water_in = Water(T=Q_(7, 'degC'), P=Q_(2, 'bar'))

water_V_dot = Q_(853.02, 'L / hr')
water_m_dot = water_in.rho * water_V_dot

Now, we write a function that returns the performance of the cooling coil for a given state of air entering the cooling coil:

In [4]:
def air_coil_fun(air_in: HumidAir) -> dict:
    """
    Given the state of air at the entry of the air cooling coil, returns the
    performance of the air cooling coil, while the other coil performance
    input parameters (the mass flow rate of air through the coil, the mass flow
    rate of water, and the entering water temperature) remain fixed.
    """
    air_coil.set_operating_conditions(
        air_in=air_in,
        water_in=water_in,
        air_m_dot=air_m_dot,
        water_m_dot=water_m_dot
    )
    result = air_coil.rate(eps_ini=0.5)
    return result

Next, we write a function that returns the state of air leaving the space and entering the cooling coil:

In [5]:
def space_fun(
    air_in: HumidAir,
    space_Q_dot: Quantity,
    space_SHR: Quantity
) -> HumidAir:
    """
    Given the state of air entering the space and the space load, returns
    the state of air leaving the space and entering the air-cooling coil.
    """
    space = AirConditioningProcess(
        air_in=air_in,
        m_da=air_m_dot,
        Q=space_Q_dot,
        SHR=space_SHR
    )
    return space.air_out

For this, we have represented the space as an instance of the `AirConditioningProcess` class in subpackage `hvac.air_conditioning`.

With the following function, we can calculate the state of the space air in steady-state conditions for the given space cooling load. We use a while-loop until the cooling capacity of the air-cooling coil matches closely with the space-cooling load. When this is the case, thermal equilibrium has been established and the final state of the space air, which is also entering the cooling coil, can be determined.

In [6]:
def system_fun(space_Q_dot: Quantity, space_SHR: Quantity) -> HumidAir:
    """
    Combines the air-cooling coil and the space into a single thermal system,
    and determines the steady-state operation of this system for a given space
    load (i.e., without any intervention of a room thermostat).
    At steady-state operation, the space load and the cooling coil capacity have
    reached thermal equilibrium.
    In steady-state conditions, the state of space air entering the cooling coil
    becomes constant in time. Also, the state of air leaving the cooling coil,
    and the state of water leaving the cooling coil becomes constant.

    Returns
    -------
    The state of the space air when steady-state operation has been reached.
    """
    solutions = {}
    i_max = 50
    i = 0
    dev_prev = None

    # Initial guess to start the loop:
    space_air = HumidAir(Tdb=Q_(26, 'degC'), RH=Q_(50, 'pct'))

    while i < i_max:
        # Calculate the coil performance for the current space air state:
        coil_perf = air_coil_fun(space_air)
        # Get the cooling capacity of the air-cooling coil:
        coil_Q_dot = coil_perf['Q_dot']
        # Get the state of air leaving the cooling coil:
        coil_air_out = coil_perf['air_out']
        # Check the deviation between the cooling capacity of the cooling coil
        # and the space cooling load:
        dev = abs(coil_Q_dot - space_Q_dot).to('W')
        if dev_prev is not None and abs(dev - dev_prev) < Q_(1.e-4, 'W'):
            break
        # Calculate the state of space air using the state of air leaving the
        # air-cooling coil:
        space_air = space_fun(coil_air_out, space_Q_dot, space_SHR)
        solutions[dev] = space_air
        dev_prev = dev
        i += 1
    else:
        print(f"No solution within tolerance after {i_max} iterations.")

    # Return the space air state for which the deviation is minimal:
    dev_min = min(dev for dev in solutions.keys())
    space_air = solutions[dev_min]
    return space_air

The while-loop will run until either the deviation between the calculated cooling coil capacity and the given space load has become sufficiently small (less than 1.e-4 W), or the maximum number of iterations has been reached. The state of the space air is returned for which the deviation between the calculated cooling coil capacity and the given space load was smallest. 

Inside `main()` we call the calculation routine `system_fun()` and then display the results.

In [7]:
def main(space_Q_dot: Quantity, space_SHR: Quantity):
    space_air = system_fun(space_Q_dot, space_SHR)

    coil_perf = air_coil_fun(space_air)
    coil_air_out = coil_perf['air_out']
    coil_water_out = coil_perf['water_out']
    coil_Q_dot = coil_perf['Q_dot']

    # Show the results:
    print(
        "space air state = "
        f"{space_air.Tdb.to('degC'):~P.2f} DB "
        f"{space_air.RH.to('pct'):~P.0f} RH",
        "leaving air state of cooling coil = "
        f"{coil_air_out.Tdb.to('degC'):~P.2f} DB "
        f"{coil_air_out.RH.to('pct'):~P.0f} RH",
        "leaving water temperature of cooling coil = "
        f"{coil_water_out.T.to('degC'):~P.2f}",
        "cooling coil capacity (check) = "
        f"{coil_Q_dot.to('kW'):~P.3f}",
        # should almost be equal to `space_Q_dot`
        sep='\n'
    )

We call `main()` with the total space cooling load and the sensible heat ratio of the space cooling load:  

In [8]:
if __name__ == '__main__':
    main(
        space_Q_dot=Q_(4.789, 'kW'),
        space_SHR=0.75
    )

space air state = 27.44 °C DB 44 % RH
leaving air state of cooling coil = 15.09 °C DB 78 % RH
leaving water temperature of cooling coil = 11.82 °C
cooling coil capacity (check) = 4.793 kW
